In [369]:
# !pip install tqdm

In [423]:
import pandas as pd
import tarfile
from tqdm import tqdm, tqdm_notebook, tnrange
import json
import numpy as np
import time
from copy import deepcopy

In [424]:
## The restaurants in the five candidate cities
df_res_bus = pd.read_csv('../datasets/filter_data_sando/tucson_business.csv', index_col= None)
### reviews in the top ten candidate cities
reviews = pd.read_csv('../datasets/filter_data_sando/tucson_reviews.csv')
users = pd.read_csv('../datasets/filter_data_sando/tucson_users.csv')

In [425]:
df_res_bus = df_res_bus.rename(columns={"name":"restaurant"})

In [426]:
df_res_bus.head()

,business_id,restaurant,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,f82dhKNiUXsDVPMLqKYiIQ,Sher-e-Punjab,853 East Grant Rd,Tucson,AZ,85719,32.250960,-110.959158,4.0,446,1,"{'RestaurantsAttire': ""'casual'"", 'BusinessAcc...","Restaurants, Salad, Pakistani, Indian, Cocktai...","{'Tuesday': '16:0-21:0', 'Wednesday': '16:0-21..."
1,Y3ZCO17N1_T_Ms1JmswwzA,Taqueria Pico De Gallo,2618 S 6th Ave,Tucson,AZ,85713,32.192017,-110.968540,4.0,397,1,"{'RestaurantsGoodForGroups': 'True', 'Caters':...","Mexican, Restaurants","{'Monday': '9:0-21:0', 'Tuesday': '9:0-21:0', ..."
2,lhsQkb5nhf-Kd5OvgB9MNg,Sachiko Sushi,3210 E Valencia Rd,Tucson,AZ,85706,32.133679,-110.923443,3.5,191,1,"{'RestaurantsPriceRange2': '2', 'BusinessAccep...","Japanese, Restaurants, Sushi Bars","{'Monday': '11:0-20:0', 'Tuesday': '11:0-20:0'..."
3,XyGRDrgCK0z4CiA6nhwEaw,Persian Room Fine Dining,"9290 N Thornydale Rd, Ste 100",Tucson,AZ,85742,32.375016,-111.046320,4.5,258,1,"{'Alcohol': ""'full_bar'"", 'GoodForMeal': ""{'de...","Restaurants, Persian/Iranian, Mediterranean","{'Monday': '11:0-21:30', 'Tuesday': '11:0-21:3..."
4,-3-6BB10tIWNKGEF0Es2BA,The 81 Hong Kong Cafe,"625 E Wetmore Rd, Ste 109",Tucson,AZ,85705,32.288540,-110.963144,4.0,133,1,"{'Alcohol': ""u'none'"", 'DogsAllowed': 'False',...","Coffee & Tea, Asian Fusion, Food, Bubble Tea, ...","{'Monday': '0:0-0:0', 'Wednesday': '10:0-21:0'..."


In [427]:
reviews = reviews.rename(columns={"stars":"rating"})


In [428]:
reviews.head()

,review_id,user_id,business_id,rating,useful,funny,cool,text,date
0,UBp0zWyH60Hmw6Fsasei7w,4Uh27DgGzsp6PqrH913giQ,otQS34_MymijPTdNBoBdCw,4,0,2,0,The bun makes the Sonoran Dog. It's like a snu...,2011-10-27 17:12:05
1,ymhbOMW63B_vGaRFR3XT0A,yZdAhQ_KmKuCZmbBy9YDQg,5Ce3lZksYVkCbrihqylVHQ,5,0,0,0,I just started going to Sushi Nara this month ...,2014-07-25 17:56:26
2,ypFqmURIY41F4pWaMW1VrQ,hToW2eoTAYlOcuNz7I73Cg,7L1kXfwU5XM6f6Jxo_L7Yg,5,1,0,0,We've been coming here since I was a kid. The ...,2015-07-03 22:36:04
3,M7h8K7PrLrN8yQb3RhHLdA,CysTz612pTix75Fye6-D4Q,WLiqfxv_GhFFA5sm878a2w,5,0,1,1,Blew $80 on food for the family was worth it f...,2013-12-29 03:15:34
4,_u331rYIp7qJ7aWvLup9Ug,BUpAhzsbbKbTqyBjT8YnNQ,wa_bwyY57etHjtJ2Fw0E3g,5,0,0,0,Called in a Mediterranean chicken shawarma. Go...,2014-04-01 05:00:09


In [429]:
#number of unique users and restaurants
n_users = len(reviews.user_id.unique())
n_restaurants = len(reviews.business_id.unique())
print('Unique Users: {0}, unique restaurants: {1}'.format(n_users, n_restaurants))

Unique Users: 68784, unique restaurants: 702


In [430]:
users.head()

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,AUi8MPWJ0mLkMfwbui27lg,John,109,2010-01-07 18:32:04,154,20,23,NaN,"gy5fWeSv3Gamuq9Ox4MV4g, lMr3LWU6kPFLTmCpDkACxg...",4,...,0,0,0,0,1,6,3,3,0,0
1,x7YtLnBW2dUnrrpwaofVQQ,Ronskee,37,2010-05-06 00:40:56,56,29,29,NaN,"EPBLDry-ObheloH-NuvbvQ, 0GHsxjCk-Ynzx4Ihj8tjPQ...",9,...,1,0,0,0,12,33,66,66,1,12
2,XLs_PhrJ7Qwn_RfgMM7Djw,Weili,90,2009-07-12 14:34:54,71,21,19,NaN,"gTX68_Us2eI2KjY21aOQ7g, e4nHt3ErihX7II-MHXVM1w...",1,...,0,0,0,0,1,3,0,0,0,0
3,QJI9OSEn6ujRCtrX06vs1w,J,1982,2007-03-19 18:15:32,14004,6466,9879,2010,"RyPeT_ICAtX8ah9dhDpEFw, W8r4aKPZFT3GPIQQDbqB6Q...",316,...,98,156,52,68,786,3674,2093,2093,562,1509
4,KXZ8A8h7Q9ZJRU5TM9asjw,Joanne,102,2010-03-20 17:40:40,150,42,30,NaN,"2N4UPgpSHFYJ1OZ0WcvTEQ, JgZ1-9Ii2wCzHE2ylZb6qQ...",7,...,0,0,0,0,5,6,2,2,0,2


Train Test Split

In [431]:
SAMPLING_RATE = 1/5

In [432]:
# Downsample by users
user_id_unique = reviews.user_id.unique()
user_id_sample = pd.DataFrame(user_id_unique, columns=['unique_user_id']) \
    .sample(frac= SAMPLING_RATE, replace=False, random_state=1)

reviews_sample = reviews.merge(user_id_sample, left_on='user_id', right_on='unique_user_id') \
   .drop(['unique_user_id'], axis=1)
reviews_sample.head()
#print(reviews_sample.shape)

,review_id,user_id,business_id,rating,useful,funny,cool,text,date
0,chtDBb_3unH0Wcs3W2Iahw,gSp5C3BfIVAHdH27rzoCDQ,RhuYqAT45uIR2NNdDsolXQ,5,1,0,1,Some of the finest Mexican food I've had. The ...,2017-07-18 18:47:24
1,PeI3zaEKhNS2lEya-HgiXA,gSp5C3BfIVAHdH27rzoCDQ,U3aNQ5DsABPYyJQolsCbJg,5,2,1,1,A friend took me here for a birthday dinner. T...,2016-10-15 04:45:14
2,XImtk866ERU81M8uDJH0xw,gSp5C3BfIVAHdH27rzoCDQ,WbA5ud4InNWkizW7HE5kRQ,5,0,0,0,Pence is a great place for a casual dinner or ...,2016-11-06 06:14:08
3,fEQX7X9EWsZXYJnRRsQzvw,gSp5C3BfIVAHdH27rzoCDQ,sr5A1frZiTRediG8FLEkdw,1,3,1,0,I live near this restaurant and decided to try...,2016-10-22 16:26:36
4,o3ecI64RRJq47Crn7l9esg,gSp5C3BfIVAHdH27rzoCDQ,AVBqx8FaXza6x-bb_FLOTQ,4,0,0,0,Anita's is excellent! The popovers are very go...,2017-07-18 18:44:39


In [433]:
#number of unique users and restaurants
n_users = len(reviews_sample.user_id.unique())
n_restaurants = len(reviews_sample.business_id.unique())
print('Unique Users: {0}, unique restaurants: {1}'.format(n_users, n_restaurants))

Unique Users: 13757, unique restaurants: 702


In [434]:
# hold out last review
reviews_user_date = reviews_sample.loc[:, ['user_id', 'date']]
reviews_user_date.date = pd.to_datetime(reviews_user_date.date)
index_holdout = reviews_user_date.groupby(['user_id'], sort=False)['date'].transform(max) == reviews_user_date['date']
reviews_holdout_ = reviews_sample[index_holdout]
reviews_traincv_ = reviews_sample[~index_holdout]

reviews_user_date = reviews_traincv_.loc[:, ['user_id', 'date']]
index_holdout = reviews_user_date.groupby(['user_id'], sort=False)['date'].transform(max) == reviews_user_date['date']
reviews_cv_ = reviews_traincv_[index_holdout]
reviews_train_ = reviews_traincv_[~index_holdout]

# remove the user that has fake reviews 

cv_users_del = set(reviews_cv_.user_id) - set(reviews_train_.user_id)
holdout_users_del = set(reviews_holdout_.user_id) - set(reviews_train_.user_id)
reviews_cv_ = reviews_cv_[~reviews_cv_.user_id.isin(cv_users_del)]
reviews_holdout_ = reviews_holdout_[~reviews_holdout_.user_id.isin(holdout_users_del)]

# ratings_cv_ = ratings_cv_[~ratings_cv_.user_id.isin(['HiT9sg9pvDiEVMFHJYihXg'])]
# ratings_holdout_ = ratings_holdout_[~ratings_holdout_.user_id.isin(['HiT9sg9pvDiEVMFHJYihXg'])]

print('There are {0} rows, {1} columns in training set.'.format(reviews_train_.shape[0], reviews_train_.shape[1]))
print('There are {0} rows, {1} columns in training set.'.format(reviews_cv_.shape[0], reviews_cv_.shape[1]))
print('There are {0} rows, {1} columns in holdout set.'.format(reviews_holdout_.shape[0], reviews_holdout_.shape[1]))


There are 19947 rows, 9 columns in training set.
There are 3109 rows, 9 columns in training set.
There are 3109 rows, 9 columns in holdout set.


In [435]:
reviews_cv_2=reviews_cv_
reviews_cv_2.head()

,review_id,user_id,business_id,rating,useful,funny,cool,text,date
8,ESs4GTLIog0tk0KkKjHmwA,gSp5C3BfIVAHdH27rzoCDQ,3qsHL1xuMPlPqQciHNYkeg,4,0,0,0,"First off, I didn't have the pizza or dine in....",2018-02-18 05:54:44
19,NB0K34PopuobnDs10MnboQ,qPORuflcC27780nm7s2UUg,9zlIJ7Q5W4AENjpGgaNSsQ,1,2,2,0,So the person/s making the pizza last Wednesda...,2016-11-06 03:02:00
43,joh0gpkfhXckO3t6rNJ7GA,w8xOsHgh3xi4pHKO1Fs5Dw,THoNnFJFVzzjz7hyOIQFkQ,4,2,0,1,Taking a look at the menu it seems that one ca...,2017-01-09 03:00:22
51,94CkieRKDzO0kHH4YuvY1A,d_fc9n5mQFFdoUxrOSeMAQ,muxda1cSVtplETqTfYVgZA,5,0,0,0,"Open a month, they've gotten as good as the re...",2011-03-14 22:02:15
102,7RAbIaA1BmP9-Z4bKRTkfA,J0HY6uvoRL2Rh7bc8mvfqA,eGi7CE2YaN6YMiGyXNi8XQ,5,1,0,0,"This was my first time at Spice Garden, and wo...",2021-10-17 16:44:38


In [436]:
# check if we have a enough user sample size (> 3000)
number_of_unique_users = len(reviews_train_.user_id.unique())
print(number_of_unique_users)

3109


Evaluation Metrics

In [440]:
def process(df):
    df['date']  = pd.to_datetime(df['date'])
    df['week_day'] = df['date'].dt.weekday
    df['month'] = df['date'].dt.month
    df['hour'] = df['date'].dt.hour
    df = df.merge(users, on = 'user_id')
    df = df.merge(df_res_bus, on = 'business_id')
    rename_dict = {'business_longitude': 'longitude', 'business_latitude': 'latitude',
                   'business_state':'state','business_city':'city', 'business_address': 'address'}
    df = df.rename(columns = rename_dict)
    return df

reviews_train = process(reviews_train_.copy())
reviews_holdout = process(reviews_holdout_.copy())
reviews_val = process(reviews_cv_.copy())

In [441]:
reviews_train_final = reviews_train.append(reviews_val)
reviews_entire_df = reviews_train.append(reviews_val).append(reviews_holdout)

C:\Users\joaqu\AppData\Local\Temp\ipykernel_5680\2588244626.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reviews_train_final = reviews_train.append(reviews_val)
C:\Users\joaqu\AppData\Local\Temp\ipykernel_5680\2588244626.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reviews_entire_df = reviews_train.append(reviews_val).append(reviews_holdout)
C:\Users\joaqu\AppData\Local\Temp\ipykernel_5680\2588244626.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reviews_entire_df = reviews_train.append(reviews_val).append(reviews_holdout)


In [442]:
reviews_entire_df.head()


,review_id,user_id,business_id,rating,useful_x,funny_x,cool_x,text,date,week_day,...,state,postal_code,latitude,longitude,stars,review_count_y,is_open,attributes,categories,hours
0,chtDBb_3unH0Wcs3W2Iahw,gSp5C3BfIVAHdH27rzoCDQ,RhuYqAT45uIR2NNdDsolXQ,5,1,0,1,Some of the finest Mexican food I've had. The ...,2017-07-18 18:47:24,1,...,AZ,85745,32.22999,-110.987571,4.0,207,1,"{'RestaurantsAttire': ""u'casual'"", 'Restaurant...","Restaurants, Mexican","{'Monday': '8:0-19:0', 'Tuesday': '8:0-19:0', ..."
1,-TeVNybQzUNYV9pkKUALBA,dR5K7ybTFxf-PobaCwKQXg,RhuYqAT45uIR2NNdDsolXQ,3,1,0,0,As far as tortillas go they're definitely 4 st...,2012-09-30 14:02:26,6,...,AZ,85745,32.22999,-110.987571,4.0,207,1,"{'RestaurantsAttire': ""u'casual'"", 'Restaurant...","Restaurants, Mexican","{'Monday': '8:0-19:0', 'Tuesday': '8:0-19:0', ..."
2,pPwyoEMDEQ5U9koJ8bF9Dg,PAc93PtEbYDtytBQ9Dyjug,RhuYqAT45uIR2NNdDsolXQ,3,1,0,1,"I loved visiting this west part of Tucson, ne...",2010-10-11 23:46:07,0,...,AZ,85745,32.22999,-110.987571,4.0,207,1,"{'RestaurantsAttire': ""u'casual'"", 'Restaurant...","Restaurants, Mexican","{'Monday': '8:0-19:0', 'Tuesday': '8:0-19:0', ..."
3,UjBziQ2hYBtDMQZxp-_3AQ,BeMrTqigcLyK6LnVj5pugw,RhuYqAT45uIR2NNdDsolXQ,5,1,0,0,Aka St. Marys Mexican Food. \n\nThis place has...,2014-10-21 21:41:33,1,...,AZ,85745,32.22999,-110.987571,4.0,207,1,"{'RestaurantsAttire': ""u'casual'"", 'Restaurant...","Restaurants, Mexican","{'Monday': '8:0-19:0', 'Tuesday': '8:0-19:0', ..."
4,TaiGNDz0vE8lVNVu08SFZQ,F76i3Syqc_jTR2m2vXGwuQ,RhuYqAT45uIR2NNdDsolXQ,4,1,0,0,"Finally, there service has come around. As I ...",2015-02-04 15:00:47,2,...,AZ,85745,32.22999,-110.987571,4.0,207,1,"{'RestaurantsAttire': ""u'casual'"", 'Restaurant...","Restaurants, Mexican","{'Monday': '8:0-19:0', 'Tuesday': '8:0-19:0', ..."


In [443]:
unique_city_businesses = reviews_entire_df[['city','business_id']].drop_duplicates()
unique_cities = unique_city_businesses.groupby('city').count()['business_id']
#unique_cities = unique_cities[unique_cities > 100]
out = pd.DataFrame()
for city in unique_cities.index:
    tmp = reviews_holdout[(reviews_holdout['city'] ==city) &
                          (reviews_holdout['rating'] >reviews_holdout['average_stars'])]
    if len(tmp['user_id'].unique())>4:

        ###this weird sampling technique is to ensure we dont' sample the same user twice in a same city
        five_users = np.random.choice(tmp['user_id'].unique(),5, replace = False)
        row = tmp[tmp['user_id'].isin(five_users)].groupby('user_id', group_keys=False).apply(lambda df: df.sample(1))
        out = out.append(row)

C:\Users\joaqu\AppData\Local\Temp\ipykernel_5680\1404869436.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out = out.append(row)


In [388]:
predict_df = out[['user_id','city','state']]
predict_df = predict_df.merge(unique_city_businesses, on = 'city')
predict_df.to_csv('../datasets/filter_data_sando/metric_sample.csv')

In [389]:
# random initialization, needs to be substituted by actual predictions later.
predict_df['predictions'] = 2.5

In [390]:
def get_all_metrics(predict_df, validation_subsample, ratings_train_final):
    top_10_recs = predict_df.groupby(['user_id','city'])['predictions'].nlargest(10).reset_index()
    out = validation_subsample
    cnt =0
    serendipity = 0


    for row in out.iterrows():
        row_values = row[1]
        top_10 = predict_df.loc[top_10_recs[top_10_recs['user_id'] == row_values['user_id']].level_2]['business_id']
        ###In top 10
        if row_values['business_id'] in top_10.values:
            cnt+=1
        user_history = ratings_train_final[ratings_train_final['user_id'] == row_values['user_id']]
        been_there = [i for i in top_10.values if i in  user_history.business_id.values]
        serendipity += 1-len(been_there)/10

    top_10 = cnt/len(out)
    serendipity = serendipity/len(out)

    predict_df = predict_df.reset_index()

    analysis_df = predict_df.merge(top_10_recs, left_on = ['user_id','city','index'], \
                                   right_on = ['user_id','city','level_2'])

    coverage = (analysis_df.groupby('city')['business_id'].nunique()/50).values.mean()

    predict_df['rankings']=predict_df.groupby(['city','user_id'])['predictions']. \
        rank(method="first",ascending = False)
    running_rankings =0
    for row in out.iterrows():
        row_values = row[1]
        user_recs = predict_df[(predict_df['user_id']==row_values['user_id'])
                               &(predict_df['city']==row_values['city'])
                               & (predict_df['business_id']==row_values['business_id'])
                               ]
        assert len(user_recs)==1
        running_rankings += user_recs['rankings'].sum()

    avg_rank = running_rankings / len(out)
    print(top_10, coverage, serendipity, avg_rank)

    return top_10, coverage, serendipity, avg_rank


Baseline models
Baseline 1: Bias Baseline

In [391]:
#!pip install scikit-surprise

In [392]:
from surprise import SVD
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import GridSearchCV
from surprise import Dataset
from surprise import BaselineOnly

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

In [393]:
def process(df):
    df['date']  = pd.to_datetime(df['date'])
    df['week_day'] = df['date'].dt.weekday
    df['month'] = df['date'].dt.month
    df['hour'] = df['date'].dt.hour
    df = df.merge(users, on = 'user_id')
    df = df.merge(df_res_bus, on = 'business_id')
    rename_dict = {'business_longitude': 'longitude', 'business_latitude': 'latitude',
                   'business_state':'state','business_city':'city', 'business_address': 'address'}
    df = df.rename(columns = rename_dict)
    return df

reviews_train = process(reviews_train_.copy())
reviews_test = process(reviews_holdout_.copy())
reviews_val = process(reviews_cv_.copy())

In [394]:
# remove observations that may cause cold-start problem, which breaks the model.
reviews_test = reviews_test.loc[reviews_test.business_id.isin(reviews_train.business_id)]
reviews_val = reviews_val.loc[reviews_val.business_id.isin(reviews_train.business_id)]

In [395]:
trainset = reviews_train.loc[:,['user_id', 'business_id', 'rating']]
trainset.columns = ['userID', 'itemID','rating']
valset = reviews_val.loc[:, ['user_id', 'business_id', 'rating']]
valset.columns = ['userID', 'itemID','rating']
testset = reviews_holdout.loc[:, ['user_id', 'business_id', 'rating']]
testset.columns = ['userID', 'itemID','rating']

In [396]:
reader = Reader(rating_scale = (0.0, 5.0))
train_data = Dataset.load_from_df(trainset[['userID','itemID','rating']], reader)
val_data = Dataset.load_from_df(valset[['userID','itemID','rating']], reader)
test_data = Dataset.load_from_df(testset[['userID','itemID','rating']], reader)

train_sr = train_data.build_full_trainset()
val_sr_before = val_data.build_full_trainset()
val_sr = val_sr_before.build_testset()
test_sr_before = test_data.build_full_trainset()
test_sr = test_sr_before.build_testset()

In [397]:
bsl_options = {'method': 'als', 'n_epochs':3}
bias_baseline = BaselineOnly(bsl_options)
bias_baseline.fit(train_sr)
predictions = bias_baseline.test(val_sr)
accuracy.rmse(predictions)

Estimating biases using als...
RMSE: 1.2524


1.252377613986122

In [398]:
bsl_options = {'method': 'als', 'n_epochs':5}
bias_baseline = BaselineOnly(bsl_options)
bias_baseline.fit(train_sr)
predictions = bias_baseline.test(val_sr)
accuracy.rmse(predictions)

Estimating biases using als...
RMSE: 1.2524


1.2523559556843937

In [399]:
bsl_options = {'method': 'als', 'n_epochs':9}
bias_baseline = BaselineOnly(bsl_options)
bias_baseline.fit(train_sr)
predictions = bias_baseline.test(val_sr)
accuracy.rmse(predictions)

Estimating biases using als...
RMSE: 1.2524


1.2523547149833716

Baseline 2: Collaborative Filtering via SVD

In [400]:
import matplotlib.pyplot as plt
import json
from tqdm import tqdm

In [401]:
def process(df):
    df['date']  = pd.to_datetime(df['date'])
    df['week_day'] = df['date'].dt.weekday
    df['month'] = df['date'].dt.month
    df['hour'] = df['date'].dt.hour
    df = df.merge(users, on = 'user_id')
    df = df.merge(df_res_bus, on = 'business_id')
    rename_dict = {'business_longitude': 'longitude', 'business_latitude': 'latitude',
                   'business_state':'state','business_city':'city', 'business_address': 'address'}
    df = df.rename(columns = rename_dict)
    return df

reviews_train = process(reviews_train_.copy())
reviews_holdout = process(reviews_holdout_.copy())
reviews_val = process(reviews_cv_.copy())

In [402]:
# remove observations that may cause cold-start problem, which breaks the model.
reviews_test = reviews_test.loc[reviews_test.business_id.isin(reviews_train.business_id)]
reviews_val = reviews_val.loc[reviews_val.business_id.isin(reviews_train.business_id)]

In [403]:
trainset = reviews_train.loc[:,['user_id', 'business_id', 'rating']]
trainset.columns = ['userID', 'itemID','rating']
valset = reviews_val.loc[:, ['user_id', 'business_id', 'rating']]
valset.columns = ['userID', 'itemID','rating']
testset = reviews_holdout.loc[:, ['user_id', 'business_id', 'rating']]
testset.columns = ['userID', 'itemID','rating']

In [404]:
reader = Reader(rating_scale = (0.0, 5.0))
train_data = Dataset.load_from_df(trainset[['userID','itemID','rating']], reader)
val_data = Dataset.load_from_df(valset[['userID','itemID','rating']], reader)
test_data = Dataset.load_from_df(testset[['userID','itemID','rating']], reader)

train_sr = train_data.build_full_trainset()
val_sr_before = val_data.build_full_trainset()
val_sr = val_sr_before.build_testset()
test_sr_before = test_data.build_full_trainset()
test_sr = test_sr_before.build_testset()

Hyperparameter Tuning

In [405]:
RMSE_tune = {}
n_epochs = [10, 20, 30]  # the number of iteration of the SGD procedure
lr_all = [0.001, 0.003, 0.005] # the learning rate for all parameters
reg_all =  [0.02, 0.05, 0.1, 0.4, 0.5] # the regularization term for all parameters

for n in n_epochs:
    for l in lr_all:
        for r in reg_all:
            print('Fitting n: {0}, l: {1}, r: {2}'.format(n, l, r))
            algo = SVD(n_epochs = n, lr_all = l, reg_all = r)
            algo.fit(train_sr)
            predictions = algo.test(val_sr)
            RMSE_tune[n,l,r] = accuracy.rmse(predictions)

Fitting n: 10, l: 0.001, r: 0.02
RMSE: 1.3098
Fitting n: 10, l: 0.001, r: 0.05
RMSE: 1.3119
Fitting n: 10, l: 0.001, r: 0.1
RMSE: 1.3104
Fitting n: 10, l: 0.001, r: 0.4
RMSE: 1.3123
Fitting n: 10, l: 0.001, r: 0.5
RMSE: 1.3120
Fitting n: 10, l: 0.003, r: 0.02
RMSE: 1.2726
Fitting n: 10, l: 0.003, r: 0.05
RMSE: 1.2750
Fitting n: 10, l: 0.003, r: 0.1
RMSE: 1.2758
Fitting n: 10, l: 0.003, r: 0.4
RMSE: 1.2768
Fitting n: 10, l: 0.003, r: 0.5
RMSE: 1.2782
Fitting n: 10, l: 0.005, r: 0.02
RMSE: 1.2584
Fitting n: 10, l: 0.005, r: 0.05
RMSE: 1.2595
Fitting n: 10, l: 0.005, r: 0.1
RMSE: 1.2576
Fitting n: 10, l: 0.005, r: 0.4
RMSE: 1.2635
Fitting n: 10, l: 0.005, r: 0.5
RMSE: 1.2640
Fitting n: 20, l: 0.001, r: 0.02
RMSE: 1.2880
Fitting n: 20, l: 0.001, r: 0.05
RMSE: 1.2851
Fitting n: 20, l: 0.001, r: 0.1
RMSE: 1.2869
Fitting n: 20, l: 0.001, r: 0.4
RMSE: 1.2898
Fitting n: 20, l: 0.001, r: 0.5
RMSE: 1.2913
Fitting n: 20, l: 0.003, r: 0.02
RMSE: 1.2530
Fitting n: 20, l: 0.003, r: 0.05
RMSE: 1.2522


In [406]:
import operator
min(RMSE_tune.items(), key=operator.itemgetter(1))[0]

(30, 0.005, 0.1)

Evaluation

In [407]:
# train and test on the optimal parameter
start_time = time.time()
algo_real = SVD(n_epochs = 30, lr_all = 0.005, reg_all = 0.1)
algo_real.fit(train_sr)
predictions = algo_real.test(test_sr)

In [408]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 1.2976930141448975 seconds ---


In [409]:
accuracy.rmse(predictions)

RMSE: 1.2955


1.29551914156122

In [410]:
accuracy.mae(predictions)

MAE:  1.0604


1.0604430877103996

In [411]:
r2_score([t[2] for t in predictions], [t[3] for t in predictions])

0.14527842428231563

In [444]:
# To evaluate coverage and serendipity metrics, use evaluation set created earlier.
predict_df_20 = pd.read_csv('../datasets/filter_data_sando/metric_sample.csv', index_col=0)
predict_df_20['predictions'] = 2.5 # fill in this value temporally
eval_20 = Dataset.load_from_df(predict_df_20[['user_id','business_id','predictions']], reader)

In [445]:
predict_df_20.head(20)

,user_id,city,state,business_id,predictions
0,xskgAGdEn4KrItyijRsNLQ,Tucson,AZ,RhuYqAT45uIR2NNdDsolXQ,2.5
1,xskgAGdEn4KrItyijRsNLQ,Tucson,AZ,U3aNQ5DsABPYyJQolsCbJg,2.5
2,xskgAGdEn4KrItyijRsNLQ,Tucson,AZ,WbA5ud4InNWkizW7HE5kRQ,2.5
3,xskgAGdEn4KrItyijRsNLQ,Tucson,AZ,sr5A1frZiTRediG8FLEkdw,2.5
4,xskgAGdEn4KrItyijRsNLQ,Tucson,AZ,AVBqx8FaXza6x-bb_FLOTQ,2.5
5,xskgAGdEn4KrItyijRsNLQ,Tucson,AZ,UvDud4vaI7l_JlBV3E1mHg,2.5
6,xskgAGdEn4KrItyijRsNLQ,Tucson,AZ,4QMIJJWQOh7zpEwOXhFJqw,2.5
7,xskgAGdEn4KrItyijRsNLQ,Tucson,AZ,V9XlikTxq0My4gE8LULsjw,2.5
8,xskgAGdEn4KrItyijRsNLQ,Tucson,AZ,f82dhKNiUXsDVPMLqKYiIQ,2.5
9,xskgAGdEn4KrItyijRsNLQ,Tucson,AZ,wa_bwyY57etHjtJ2Fw0E3g,2.5


In [446]:
resultado=top10.to_frame()
print(type(resultado))

<class 'pandas.core.frame.DataFrame'>


In [457]:
df_join=resultado.merge(df_res_bus[["business_id","restaurant"]],how="inner",on="business_id")
df_join.head(10)

,business_id,predictions,restaurant
0,-0G_6-KFGpCpxTUlVXCMYQ,2.5,Gio Taco
1,-1MhPXk1FglglUAmuPLIGg,2.5,Red Garter Saloon
2,-3-6BB10tIWNKGEF0Es2BA,2.5,The 81 Hong Kong Cafe
3,-OX0MJDPRHV0RCRvwYnvBQ,2.5,NI HAO TEA
4,-SZ-p4o6gJPOJ--HA3NKcw,2.5,Nimbus Bistro
5,-TzFN6nWvbVRARY2B11Epg,2.5,Mimi's Cafe
6,-iGzevsRYAi990-Lt-iMoQ,2.5,US Fries
7,-lHpvtlMkRu0oRIXJkystA,2.5,Bistro 44
8,-pRU97z3uPU_M7yUf5yzMg,2.5,Micha's
9,-xtnwq4VBA2XFobjDGz0Ww,2.5,Red Lobster


In [459]:
df_join.sort_values(by="predictions", ascending=False)

,business_id,predictions,restaurant
0,-0G_6-KFGpCpxTUlVXCMYQ,2.5,Gio Taco
471,fzTeOVjzdCo9zQopk5mB4A,2.5,Guillermo's Double L Restaurant
463,f82dhKNiUXsDVPMLqKYiIQ,2.5,Sher-e-Punjab
464,f9eWh7vqRryHRS4-H6gOlw,2.5,Kneaders Bakery & Cafe
465,fKXU_HMU09PJfTHjtInN0Q,2.5,BJ's Restaurant & Brewhouse
...,...,...,...
235,Klpb4jqrgCBX9_BnBmkz8g,2.5,BK Tacos
236,KmZuIc9o_PL2BW8j_7gy7w,2.5,Reforma Modern Mexican Mezcal + Tequila
237,Ktfl6gzqp4kWBYCpa2r35g,2.5,On A Roll
238,KuVtxfIJu4qRscngMsVcxg,2.5,Fini's Landing


In [462]:
eval_before_20 = eval_20.build_full_trainset()
eval_sr_20 = eval_before_20.build_testset()
eval_pred_20 = algo_real.test(eval_sr_20)

baseline_20 = pd.DataFrame(eval_pred_20, columns = ['userId','business_id','rating','pred_rating','x'])
predict_df_20['predictions'] = baseline_20.pred_rating

In [463]:
top_10, coverage, serendipity, avg_rank = get_all_metrics(predict_df_20, out, reviews_train_final)

AssertionError: 

In [464]:
print(baseline_20)

                      userId             business_id  rating  pred_rating  \
0     xskgAGdEn4KrItyijRsNLQ  RhuYqAT45uIR2NNdDsolXQ     2.5     3.339675   
1     xskgAGdEn4KrItyijRsNLQ  U3aNQ5DsABPYyJQolsCbJg     2.5     3.810847   
2     xskgAGdEn4KrItyijRsNLQ  WbA5ud4InNWkizW7HE5kRQ     2.5     3.385708   
3     xskgAGdEn4KrItyijRsNLQ  sr5A1frZiTRediG8FLEkdw     2.5     2.563487   
4     xskgAGdEn4KrItyijRsNLQ  AVBqx8FaXza6x-bb_FLOTQ     2.5     4.086860   
...                      ...                     ...     ...          ...   
3505  xOF3dntWsLtVbSpqvOfNMQ  eGi7CE2YaN6YMiGyXNi8XQ     2.5     4.107107   
3506  xOF3dntWsLtVbSpqvOfNMQ  kbjbMffGvyjIdJivotkIiw     2.5     3.621570   
3507  xOF3dntWsLtVbSpqvOfNMQ  f6y6grP56Rdsx0jrdnvWeA     2.5     4.058935   
3508  xOF3dntWsLtVbSpqvOfNMQ  L7J8aZmdz_DNhEwFb3QnBQ     2.5     4.191426   
3509  xOF3dntWsLtVbSpqvOfNMQ  gowa-c0fpgvosQnRWF9V3g     2.5     3.530077   

                              x  
0     {'was_impossible': False}  
1     {

In [465]:
# Define a custom sorting key based on the rating
def pred_rating (baseline_20):
    return baseline_20["pred_rating"]

In [466]:
top10_2=baseline_20.groupby('business_id')["pred_rating"].mean()
top10_2.sort_values(ascending=False).head(10)

business_id
jh8j-DWqgWkbRe_a2XtKFQ    4.852592
CJoO4HYD0tZRXlZqA04wmw    4.792253
3qsHL1xuMPlPqQciHNYkeg    4.789104
6OwxdpajDSJi3DkMqkr2sw    4.785708
vZlPeIEzNK2YkZhGcLxHOw    4.749385
DVBJRvnCpkqaYl6nHroaMg    4.743305
-OX0MJDPRHV0RCRvwYnvBQ    4.718216
2VPX222bhizLlWZrLQL3PQ    4.690496
H5nMeiXJmLpBALeV8A-FeQ    4.681259
iKLhnstKcsUILIaOHwhJ8Q    4.673738
Name: pred_rating, dtype: float64

In [467]:
resultado_2=top10_2.to_frame()
print(type(resultado_2))

<class 'pandas.core.frame.DataFrame'>


In [470]:
df_join_2=resultado_2.merge(df_res_bus[["business_id","restaurant"]],how="inner",on="business_id")
df_join_2.head(10)

,business_id,pred_rating,restaurant
0,-0G_6-KFGpCpxTUlVXCMYQ,3.866074,Gio Taco
1,-1MhPXk1FglglUAmuPLIGg,4.269565,Red Garter Saloon
2,-3-6BB10tIWNKGEF0Es2BA,3.753103,The 81 Hong Kong Cafe
3,-OX0MJDPRHV0RCRvwYnvBQ,4.718216,NI HAO TEA
4,-SZ-p4o6gJPOJ--HA3NKcw,3.654703,Nimbus Bistro
5,-TzFN6nWvbVRARY2B11Epg,4.131551,Mimi's Cafe
6,-iGzevsRYAi990-Lt-iMoQ,3.726959,US Fries
7,-lHpvtlMkRu0oRIXJkystA,3.832925,Bistro 44
8,-pRU97z3uPU_M7yUf5yzMg,3.660379,Micha's
9,-xtnwq4VBA2XFobjDGz0Ww,3.697562,Red Lobster


In [471]:
df_join_2.sort_values(by="pred_rating", ascending=False)

,business_id,pred_rating,restaurant
515,jh8j-DWqgWkbRe_a2XtKFQ,4.852592,Barrio Bread
151,CJoO4HYD0tZRXlZqA04wmw,4.792253,Crave Coffee Bar
53,3qsHL1xuMPlPqQciHNYkeg,4.789104,Anello
90,6OwxdpajDSJi3DkMqkr2sw,4.785708,Barista Del Barrio
637,vZlPeIEzNK2YkZhGcLxHOw,4.749385,Queen Sheba
...,...,...,...
94,6zxPHDdLUNhCSLSeHlCVNg,2.849871,Something Sweet Dessert Lounge
475,gIIZQRP878_yceemj2bK_A,2.822827,HI FI Kitchen & Cocktails
84,5yjHCLzwHgjKNNHgAaUC8Q,2.740331,Buffalo Wild Wings
641,vpc5PFLMo0oJQzuNZaSlyA,2.717841,Applebee's Grill + Bar
